In [5]:
ls /d/shared/silver_projects_v2/bestegg/personalloanv1/modeling/model_artifacts/personalloan/exp6.7.CO.pre_test.all_features_with2017_new_weights_foldinvalid/

app/                                    test_app.parquet
asset.json                              test_auc.json
best_model_params.json                  test_data_summary.json
calibration_object.obj                  test_fe_data.parquet
data_description.json                   test_ks.json
feature_definition.parquet              test_sample_weight.parquet
feature_importance.parquet              test_scores.parquet
fe_data/                                test_target.parquet
fit_time_info.json                      test_zest_scores.parquet
keep_features.json                      top_features.parquet
key_factors_mapping.json                train_app.parquet
model.obj                               train_auc.json
model_strategy.json                     train_data_summary.json
mrm_pipeline.obj                        train_fe_data.parquet
overall_scores/                         train_history.json
parsed_monotonic_constraints_list.json  train_ks.json
pipeline.obj                            train_s

In [6]:
base_folder = '/d/shared/silver_projects_v2/bestegg/personalloanv1/modeling/model_artifacts/personalloan/exp6.7.CO.pre_test.all_features_with2017_new_weights_foldinvalid/'

In [ ]:
import pickle
import model_engine

def load_pipe(folder, mrm = False):
    if mrm:
        location = f'{folder}mrm_pipeline.obj'
    else:
        location = f'{folder}pipeline.obj'
    print(f'MRM: {mrm} location: {location}')
    with open(location, 'rb') as f:
        meta_data = pickle.load(f)
        pipe = pickle.load(f)
    return pipe